In [ ]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'ClusterCast.settings'
import test_utils
import test_utils_models
import django
django.setup()
import sys
sys.path.append("/home/ajp031/StockDeepLearning/ClusterCast/ClusterCast")
from asgiref.sync import sync_to_async
from importlib import reload
import ClusterPipeline.models.ClusterProcessing as cp
import ClusterPipeline.models.SequencePreprocessing as sp
from copy import deepcopy
import numpy as np
from tensorflow.keras.backend import clear_session
from tslearn.metrics import dtw
import tfts
from tfts import AutoModel, KerasTrainer
import tensorflow as tf
import random

In [ ]:
reload(test_utils)

In [ ]:
group_ids = [4]
groups = await test_utils.extract_cluster_groups(group_ids)

In [ ]:
clusters = await test_utils.get_all_clusters(groups)

cur_group = groups[0]

In [ ]:
cluster_of_interest_id = 17
cluster_of_interest = [cluster for cluster in clusters if cluster.id == cluster_of_interest_id][0]

In [ ]:
features = groups[0].group_params.training_features

features

In [ ]:
X_train, y_train, X_test, y_test = cluster_of_interest.get_3d_array()


In [ ]:
print(X_test[0, :,0])
print(cluster_of_interest.test_seq_elements[0].seq_x[:,0])
print(cluster_of_interest.test_seq_elements[0].start_date)

In [ ]:
print(X_test[0, :,50])
print(cluster_of_interest.test_seq_elements[-1].seq_x[:,50])
print(cluster_of_interest.test_seq_elements[-1].start_date)

In [ ]:
print(features)

In [ ]:
target_features = [] 
# target_features += ['pctChgclose{}_target'.format(i) for i in range(-14, 0) ]
# target_features += ['pctChgclose-0_target']
target_features += ['pctChgclose+{}_target'.format(i) for i in range(1, 26) ]

training_features = []
training_features = random.sample(features, 125)
# training_features += ['pctChgvolume', 'pctChgclose','pctDiff+ema50_close','pctDiff+ema10_close','deriv+ema10' ]
# training_features += ['pctChgvolume', 'pctChgclose']
# training_features = ['pctChgclose', 'pctChgvolume','pctDiff+ema50_close','pctDiff+ema10_close','deriv+ema10', 'pctChgVix','pctDiff+bb_high_close10','pctDiff+bb_low_close10', 'bb_indicator40','pctChgClOp','pctDiff+bb_high_low10','stoch_k','pctDiff+ema10_close' ]
# training_features = "pctDiff+bb_low_close20, pctDiff+bb_high_close10, pctDiff+bb_high_low60, stoch_k, pctChgclose, deriv+sma10, hiLo, rsi10, pctDiff+sma5_sma50, pctDiff+sma10_sma20, deriv+ema20, pctDiff+ema20_ema100, pctDiff+ema30_ema50, deriv+ema30, deriv+sma200, pctDiff+sma50_sma100, pctDiff+sma20_close, opLo, pctChgvolume, pctDiff+bb_low_close60"
# training_features = "rsi100, pctDiff+bb_high_low60, pctDiff+ema5_ema200, rsi20, deriv+sma5, pctDiff+ema20_ema100, bb_indicator20, pctDiff+bb_high_low20, bb_indicator60, pctDiff+ema30_ema200, pctChglow, rsi50, stoch_d, pctDiff+sma200_close, hiLo, pctDiff+bb_high_low40, pctDiff+bb_low_close60, pctDiff+sma30_sma200, pctDiff+sma10_close, bb_indicator10"
# training_features = [item.strip() for item in training_features.split(",")]
print(training_features)
y_feature_sets = cur_group.group_params.y_feature_sets
# training_features += ['pctChgclose']


In [ ]:
# for feature_set in y_feature_sets:
#     print(feature_set)
#     print(type(feature_set))
y_feature_sets = y_feature_sets[:-1]

In [ ]:
reload(test_utils)
reload(test_utils_models)

In [ ]:
print(target_features)
print(len(target_features))

In [ ]:
y_feature_dict = cur_group.y_feature_dict
X_feature_dict = cur_group.X_feature_dict

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_feature_dict)

In [ ]:

X_train_filtered = test_utils.filter_by_features(X_train, training_features, X_feature_dict)
X_test_filtered = test_utils.filter_by_features(X_test, training_features, X_feature_dict)
y_train_filtered = test_utils.filter_y_by_features(y_train, target_features, y_feature_dict)
y_test_filtered = test_utils.filter_y_by_features(y_test, target_features, y_feature_dict)
print(X_train_filtered.shape)
print(y_train_filtered.shape)
print(X_test_filtered.shape)
print(y_test_filtered.shape)

In [ ]:
print(X_train_filtered[25, 0,:])

In [ ]:
print(y_train_filtered[25, :])

In [ ]:
model = test_utils_models.CustomSeq2SeqModel(len(target_features), X_train_filtered.shape[1], X_train_filtered.shape[2])

In [ ]:
training_model = test_utils.train_model(model, (X_train, y_train, X_test, y_test), training_features, 
                                                   target_features, X_feature_dict, y_feature_dict,
                                                   epochs = 100, batch_size = 15, lr = 0.0001, early_stopping_patience = 15,
                                                   loss = 'mse')

In [ ]:
num_days = len(target_features)
attention_model_accuracy, attention_results, predicted_y, attention_weights = test_utils.eval_model(X_test_filtered, y_test_filtered, model,num_days,test_model= True)

In [ ]:
attention_weights.shape

In [ ]:
predicted_y_transformed = np.zeros_like(predicted_y)
for i, feature in enumerate(target_features):
    scaler = [feature_set for feature_set in y_feature_sets if feature == feature_set.name][0].scaler
    predicted_y_transformed[:, i] = scaler.inverse_transform(predicted_y[:, i].reshape(-1, 1)).reshape(-1)

In [1]:
reload(test_utils)
reload(sp)
reload(test_utils_models)

NameError: name 'reload' is not defined

In [ ]:
print(attention_model_accuracy)

test_utils.visualize_future_distribution(attention_results)

In [ ]:


for i in range(20,40):
    attention_weight = attention_weights[i]
    sequence_element = cluster_of_interest.test_seq_elements[i]
    prediction = predicted_y_transformed[i]
    print(sequence_element.y_feature_sets)
    plot = test_utils.plot_attention_weights_single(attention_weight)
    # plot.show()
    sp.SequenceElement.visualize_future_movement_mpl(sequence_element, prediction, scaler, isCuma = False, target_features=target_features, num_days=num_days)